In [1]:
cd ..

/home/admin-hieunn/important/VNUIS-Chatbot


/home/admin-hieunn/important/VNUIS-Chatbot/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("./Data/DB/vnuis.csv")

In [4]:
df.head()

,Relevant docs,Domain level 1,Domain level 2,Domain level 3,Question 1,Question 2,Question 3,Question 4,Question 5,Question 6,Question 7,Question 8,Question 9,Question 10
0,"Với 20 năm xây dựng và phát triển, VNU-IS là m...",Giới thiệu chung,Đại học do ĐHQGHN cấp bẳng,Thông điệp của Hiệu Trưởng,VNU-IS có phải là một trong những đơn vị đầu t...,Nhà trường là đơn vị tiên phong như thế nào?,Nhà trường đã có hợp tác đào tạo và nghiên cứu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Lời chúc của Hiệu Trưởng: Chúc cho sự đoàn kết...,Giới thiệu chung,Đại học do ĐHQGHN cấp bẳng,Thông điệp của Hiệu Trưởng,Lời chúc của Hiệu Trưởng là gì?,Hiệu Trưởng đã có phát biểu như thế nào?,Lời chúc hướng đến mục tiêu gì của Hiệu Trưởng...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Thông tin cơ bản của Trường Quốc Tế\nTên đầy đ...,Giới thiệu chung,Đại học do ĐHQGHN cấp bẳng,Sơ lược về Trường Quốc Tế,VNU-IS có bao nhiêu cơ sở và các cơ sở này nằm...,Tên đầy đủ của Trường Quốc Tế?,Các thông tin cơ bản của Trường Quốc Tế - VNU?,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Trường Quốc tế (tên giao dịch bằng tiếng Anh V...,Giới thiệu chung,Đại học do ĐHQGHN cấp bẳng,Sơ lược về Trường Quốc Tế,Ý nghĩa tên Trường Quốc Tế?,Sứ mệnh của trường Quốc Tế?,Giới thiệu về Trường Quốc Tế?,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Trường Quốc tế - 20 năm hình thành và phát tri...,Giới thiệu chung,Đại học do ĐHQGHN cấp bẳng,Sơ lược về Trường Quốc Tế,20 năm hình thành và phát triển của Trường Quố...,Quá trình hình thành và phát triển của Trường ...,"Số lượng chương trình đào tạo, sinh viên, và c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df["relevantdocs_id"] = pd.factorize(df["Relevant docs"])[0]

In [6]:
df["relevantdocs_id"] = "relevantdocs_id" + df["relevantdocs_id"].astype(str)

In [7]:
corpus = df[["Relevant docs", "relevantdocs_id"]]

In [8]:
corpus.to_csv("./Data/finetune_embedding/corpus.csv", index = False)

In [9]:
df_final = pd.melt(
    df,
    id_vars=["relevantdocs_id"],
    value_vars=["Question 1", "Question 2", "Question 3", "Question 4", "Question 5", "Question 6", "Question 7", "Question 8", "Question 9", "Question 10"],
    var_name="Original_Question_Column",
    value_name="Question_Text",
)

In [10]:
df_final = df_final.drop(columns=["Original_Question_Column"])
df_final = df_final.rename(columns={"Question_Text": "Question"})


In [11]:
df_final.dropna(subset=["Question"], inplace=True)
# df_final.drop_duplicates(subset="Question", inplace=True, ignore_index=True)

In [12]:
df_final

,relevantdocs_id,Question
0,relevantdocs_id0,VNU-IS có phải là một trong những đơn vị đầu t...
1,relevantdocs_id1,Lời chúc của Hiệu Trưởng là gì?
2,relevantdocs_id2,VNU-IS có bao nhiêu cơ sở và các cơ sở này nằm...
3,relevantdocs_id3,Ý nghĩa tên Trường Quốc Tế?
4,relevantdocs_id4,20 năm hình thành và phát triển của Trường Quố...
...,...,...
5076,relevantdocs_id307,"Theo thang điểm 10, bao nhiêu điểm thì đạt D"
5166,relevantdocs_id396,Ca 4 bắt đầu và kết thúc vào khoảng thời gian ...
5192,relevantdocs_id422,Tên các môn học của TS. Trần Đức Quỳnh là gì?
5595,relevantdocs_id230,Câu lạc bộ dành cho sinh viên đam mê bóng đá


In [12]:
df_final["queries_id"] = df_final.index

In [13]:
df_final["queries_id"] = "queries_id" + df_final["queries_id"].astype(str)

In [14]:
queries = df_final[["queries_id", "Question"]]

In [15]:
queries.to_csv("./Data/finetune_embedding/queries.csv", index=False)

In [16]:
eval = df_final[["queries_id", "relevantdocs_id"]]

In [17]:
eval.to_csv("./Data/finetune_embedding/eval.csv", index=False)

In [18]:
eval

,queries_id,relevantdocs_id
0,queries_id0,relevantdocs_id0
1,queries_id1,relevantdocs_id1
2,queries_id2,relevantdocs_id2
3,queries_id3,relevantdocs_id3
4,queries_id4,relevantdocs_id4
...,...,...
5076,queries_id5076,relevantdocs_id307
5166,queries_id5166,relevantdocs_id396
5192,queries_id5192,relevantdocs_id422
5595,queries_id5595,relevantdocs_id230
